In [1]:
from sklearn.datasets import fetch_california_housing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
housing = fetch_california_housing(as_frame=True)
print(housing.frame)

       MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0      8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1      8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2      7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3      5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4      3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   
...       ...       ...       ...        ...         ...       ...       ...   
20635  1.5603      25.0  5.045455   1.133333       845.0  2.560606     39.48   
20636  2.5568      18.0  6.114035   1.315789       356.0  3.122807     39.49   
20637  1.7000      17.0  5.205543   1.120092      1007.0  2.325635     39.43   
20638  1.8672      18.0  5.329513   1.171920       741.0  2.123209     39.43   
20639  2.3886      16.0  5.254717   1.162264      1387.0  2.616981     39.37   

       Longitude  MedHouseVal  
0      

In [3]:
housing.frame.sample(5).round(2)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
7751,2.59,28.0,4.24,1.03,912.0,3.72,33.92,-118.15,1.61
7850,2.67,25.0,4.33,1.07,4149.0,3.05,33.89,-118.18,1.32
14755,1.34,18.0,4.82,1.08,1429.0,4.98,32.57,-117.06,0.95
20544,2.06,23.0,4.80,1.01,6330.0,3.46,38.55,-121.76,2.19
18219,5.77,26.0,5.43,1.03,815.0,3.74,37.41,-122.07,3.22


In [4]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=3)
kmeans.fit(housing.data)
군집 = kmeans.predict(housing.data)

pd.Series(군집).value_counts(normalize=True).round(2)

0    0.89
1    0.11
Name: proportion, dtype: float64

In [6]:
군집별 = housing.frame.groupby(군집)
군집별.mean().round(2)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,3.87,29.79,5.46,1.10,1129.77,2.90,35.69,-119.62,2.07
1,3.90,19.73,5.19,1.07,3707.24,4.38,35.20,-119.16,2.02


In [22]:
관심특성목록 = ['Population', 'AveOccup']
print('전체 평균')
print(housing.frame[관심특성목록].mean().round(2))
housing.frame[군집==1].sort_values(by=관심특성목록[::-1], ascending=False).round(2).head(10)

전체 평균
Population    1425.48
AveOccup         3.07
dtype: float64


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
19006,10.23,45.0,3.17,0.83,7460.0,1243.33,38.32,-121.98,1.38
3364,5.52,36.0,5.14,1.14,4198.0,599.71,40.41,-120.51,0.68
16669,4.26,46.0,9.08,1.31,6532.0,502.46,35.32,-120.70,3.50
13034,6.14,52.0,8.28,1.52,6675.0,230.17,38.69,-121.15,2.25
9172,4.24,5.0,5.12,0.93,8733.0,83.17,34.47,-118.59,1.55
13366,4.26,36.0,5.26,1.12,2886.0,33.95,33.94,-117.63,1.83
5986,1.88,52.0,4.50,1.21,2688.0,21.33,34.10,-117.71,2.12
14756,2.56,23.0,4.59,1.07,5176.0,18.82,32.56,-116.97,1.53
1039,4.30,16.0,6.09,1.12,4930.0,17.18,38.42,-120.97,1.22
7164,2.18,36.0,4.81,1.11,2862.0,15.81,34.06,-118.17,1.24


In [38]:
특성별_정상필터 = {}

for 특성명 in housing.data.columns:
    특성열 = housing.data[특성명]
    Q1, Q3 = np.percentile(특성열, [25, 75])
    IQR = Q3 - Q1
    이상치하한 = Q1 - 1.5 * IQR
    이상치상한 = Q3 + 1.5 * IQR
    특성별_정상필터[특성명] = 특성열.between(이상치하한, 이상치상한)

정상필터 = np.logical_and.reduce(list(특성별_정상필터.values()))
print(f'전체 관측치 수: {len(정상필터):,}, 정상 범위: {np.sum(정상필터):,}')

전체 관측치 수: 20,640, 정상 범위: 16,842
